<a href="https://colab.research.google.com/github/sergiofraile/when_flutter_meets_tensorflow_part_1/blob/master/mnist_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf

In [2]:
# Load the MNIST dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()

# Normalize the training values
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
# A callback to stop our tranning
# when reaching enough accuracy
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.99):
      print("\nReached 99.0% accuracy so cancelling training!")
      self.model.stop_training = True

In [4]:
# Create a basic model
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

W0820 14:14:16.301352 139794456360832 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [6]:
# Train the model
model.fit(x_train, 
          y_train, 
          epochs=25,
          callbacks=[myCallback()])

Epoch 1/25
60000/60000 [==============================] - 7s 111us/sample - loss: 0.2954 - acc: 0.9138
Epoch 2/25
60000/60000 [==============================] - 6s 98us/sample - loss: 0.1409 - acc: 0.9581
Epoch 3/25
60000/60000 [==============================] - 6s 98us/sample - loss: 0.1051 - acc: 0.9680
Epoch 4/25
60000/60000 [==============================] - 6s 98us/sample - loss: 0.0877 - acc: 0.9732
Epoch 5/25
60000/60000 [==============================] - 6s 99us/sample - loss: 0.0733 - acc: 0.9765
Epoch 6/25
60000/60000 [==============================] - 6s 98us/sample - loss: 0.0654 - acc: 0.9796
Epoch 7/25
60000/60000 [==============================] - 6s 99us/sample - loss: 0.0583 - acc: 0.9811
Epoch 8/25
60000/60000 [==============================] - 6s 98us/sample - loss: 0.0542 - acc: 0.9827
Epoch 9/25
60000/60000 [==============================] - 6s 98us/sample - loss: 0.0490 - acc: 0.9839
Epoch 10/25
60000/60000 [==============================] - 6s 99us/sample - loss:

In [7]:
# Evaluate the model
model.evaluate(x_test, y_test)

10000/10000 [==============================] - 1s 67us/sample - loss: 0.0821 - acc: 0.9800


[0.08206978416955972, 0.98]

In [0]:
# Save the model
model.save('mnist_model.h5')

In [9]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model_file('mnist_model.h5')
tflite_model = converter.convert()
open("converted_mnist_model.tflite", "wb").write(tflite_model)

W0820 14:16:07.405250 139794456360832 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0820 14:16:07.406886 139794456360832 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0820 14:16:07.987820 139794456360832 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/lite/python/util.py:238: convert_variables_to_constants (from tensorflow.python.framework.graph_util_impl) is

408152

In [0]:
# Download the tflite model
from google.colab import files

files.download('converted_mnist_model.tflite')

In [0]:
# Uncomment and use this block if you want test the model
# prediction with your own images

# import numpy as np
# from google.colab import files
# from keras.preprocessing import image
# from PIL import Image


# uploaded = files.upload()

# for fn in uploaded.keys():
#   # predicting images
#   path = fn
#   img = Image.open(path).convert("L")
#   x = image.img_to_array(img)
#   x = np.expand_dims(x, axis=0)

#   images = np.vstack([x])
#   classes = model.predict(images)
#   print(fn)
#   print(classes)